# Performance Analysis

In this analysis, we explore various computational methods to evaluate [Lily Jackson's solo performance in Paris Cavanagh's choreography](https://www.youtube.com/watch?v=YtdouPefSrQ). The following computational methods will be used: Rate of Change Trajectory, Range of Motion, Motion Energy Map, Joint Velocities, Beat Alignment, and Principal Component Analysis. In this notebook, we aim to explore ways to logically structure an analysis and what would be helpful for feature extraction.

### Requirements

In [1]:
import os
import cv2
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import numpy as np
import moten
import pandas as pd
from matplotlib.gridspec import GridSpec
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

### Load Data


In [2]:
kd = pd.read_csv('data/processed/contemporary_001.csv')
image_path = 'data/interim/contemporary_001/frames'
image_files = sorted([f for f in os.listdir(image_path) if f.endswith('.png')])
video_file_path = 'data/external/contemporary_001.mp4'

### Selecting a Time Interval
The code snippet below enables you to focus on a specific time interval within the kinematic data, allowing for a more targeted analysis of the dancer's performance.

In [3]:
kd = kd.drop(kd.columns[[0, 2]], axis=1)
kd = kd.iloc[2:34]
kd.head(5)

,frame,nose_x,nose_y,nose_z,left_eye_inner_x,left_eye_inner_y,left_eye_inner_z,left_eye_x,left_eye_y,left_eye_z,...,diff_right_knee_z,diff_left_knee_x,diff_left_knee_y,diff_left_knee_z,diff_right_ankle_x,diff_right_ankle_y,diff_right_ankle_z,diff_left_ankle_x,diff_left_ankle_y,diff_left_ankle_z
2,60,0.485273,0.219256,-0.280624,0.492189,0.202155,-0.253160,0.496962,0.202840,-0.253058,...,-0.003858,-0.009441,0.010817,0.000331,-0.016307,0.037791,0.015470,-0.012651,0.042522,0.035923
3,90,0.486556,0.216945,-0.248385,0.493592,0.200263,-0.220545,0.498182,0.200760,-0.220472,...,0.003958,0.001870,-0.001290,0.001910,0.000252,0.004040,-0.010962,0.003609,-0.016371,-0.030781
4,120,0.433745,0.230009,-0.101605,0.439995,0.209614,-0.086546,0.444377,0.208915,-0.086575,...,0.005218,-0.000306,-0.003044,-0.069447,0.004292,0.010251,0.017208,0.009121,0.044594,-0.118247
5,150,0.443140,0.251476,-0.083362,0.447163,0.233594,-0.052887,0.451157,0.233189,-0.052941,...,-0.034225,-0.002392,-0.002596,0.028837,0.002562,0.011435,-0.048514,-0.009465,-0.053983,0.061190
6,180,0.439002,0.684247,0.402012,0.433837,0.692792,0.397050,0.433185,0.697021,0.397056,...,0.019550,0.004883,-0.262904,-0.062697,0.118837,-0.519156,-0.401491,0.041640,-0.450467,-0.459375


# Rate of Change Trajectory

**Rate of Change Trajectory** helps us understand the positions of the dancer's joints in three dimensions (horizontal, vertical, and frontal) over time. By examining the trajectories, we can gain insight into the shape and spatial aspects of the dance, laying the foundation for our analysis. The following plots displays the screen caps per second and trajectory rate change between `00:02`-`00:31`. The straight lines represent the *upper body movement* and dashed lines represent *lower body movements*
* **x-joint trajectory** represents the horizontal movemen of each joint
* **y-joint trajectory** represents the vertical movement of each joint
* **z-joint trajectory** represents the distance of the performer to the camera, with negative values indicating stepping backward and positive values indicating forward movements.

In [ ]:
plt.rcParams.update({'font.size': 16})
plt.rcParams['figure.constrained_layout.use'] = True
frame_numbers = sorted(kd['frame'].tolist())
cols, rows = 10, 4
fig = plt.figure(figsize=(30, 10))
for idx, frame_number in enumerate(frame_numbers):
    row_idx, col_idx = divmod(idx, cols)
    image_file = f"{str(frame_number).zfill(5)}.png"
    img = cv2.imread(os.path.join(image_path, image_file))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ax = fig.add_subplot(rows, cols, idx + 1)
    ax.imshow(img)
    ax.set_title(f"00:{kd.index[idx]:02d}")
    ax.axis('off')
plt.show()


fig = plt.figure(figsize=(24, 12))
gs = GridSpec(rows, cols, figure=fig)
colors = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'gray', 'purple', 'orange', 'pink', 'brown']
line_styles = ['-','-','-','-','-','-',':', ':', ':',':']
diff_l = ['Forehead','Torso','R Shoulder','L Shoulder','R Elbow','L Elbow','R Knee','L Knee','R Ankle','L Ankle']
diff_xyz = [col_name for col_name in kd.columns if col_name.startswith('diff_') and col_name[-2:] in ['_x', '_y', '_z']]

for r in range(1, rows):
    ax = fig.add_subplot(gs[r, :]) 
    for i, col in enumerate(diff_xyz[r-1::3]):
        ax.plot(kd.index, kd[col], color=colors[i % len(colors)], linestyle=line_styles[i % len(line_styles)], label=diff_l[i])
    ax.set_xticks(np.arange(kd.index.min(), kd.index.max()+1, 1))
    ax.set_xlim(kd.index.min(), kd.index.max())
    ax.set_ylim(-1, 1)
    ax.set_xlabel('Time (mm:ss)') 
    ax.set_title(f"{col[-2:].upper()} Trajectory Rate Change from 00:02 to 00:31")
    ax.grid()
    
ax.legend(loc='lower center', ncol=8, bbox_to_anchor=(0.5, -1.25))
plt.tight_layout()

### Quick Observations

* A notable peaks can be seen between `00:06`-`00:07`, however, up on manual inspection using the screenshots indicated an absent landmark. Therefore, the trajectory peaks do not correspond to sudden movement changes.
* Throughout the choreography, the performer emphasizes upper body movements over lower body movements, resulting in more noticeable rate of change in the `right_shoulder`, `right_elbow`, `left_shoulder`, and `left_elbow` joints.
* With a higher rate of change found in the upper body joints, delving into the wrist, thumbs, and hand movements might be more insightful than the lower body.
* The dancer primarily remains stationary during the performance, leading to reduced activity in all **x-joint trajectories**.
* Most changes in trajectory range from `-0.2` to `0.2`. Although these variations are subtle, the cumulative effect contributes to the percussive dynamic gestures observed in the dance.

# Range of Motion

This metric measures the extent of movement for each joint throughout the dance. A wider range of motion indicates that the dancer is fully utilizing the space available to them. This builds upon the spatial understanding from the previous metric.

In [ ]:
a_cols = [col for col in kd.columns if col.startswith('a_')]

min_values = kd[a_cols].min()
max_values = kd[a_cols].max()

rom_values = max_values - min_values

kd_rom = pd.DataFrame({
    'joint': [col[2:] for col in a_cols],
    'min_angle': min_values.values,
    'max_angle': max_values.values,
    'rom': rom_values.values
})

kd_rom = kd_rom.sort_values(by='rom', ascending=False)
print(kd_rom.head(15))

### Quick Observations

The ROM (range of motion) values tell us about the variability in movement for each joint pair during the motion capture session. A higher ROM value suggests that the joint underwent more diverse movements during the session, while a lower ROM value suggests that the joint was more stable and did not move as much.

* The joint pair with the highest ROM is the "forehead_torso" pair, indicating a lot of movement in the upper body region.
* Four of the top five joint pairs with the highest ROM values are in the upper body region, suggesting that there were more movements in the upper body compared to the lower body during the motion capture session.
* The joint pairs with the lowest ROM values are in the ankle and foot regions, suggesting that these joints were relatively stable and did not move as much during the session.

## Motion Energy using Pymoten

Luminance in motion analysis measures the changes in pixel intensities between consecutive video frames. The Moten model uses a bank of spatiotemporal filters to analyze these changes and extract motion direction, speed, and orientation information. Color intensity in Moten's output represents the magnitude of motion energy, with yellow indicating high and blue indicating low motion energy. The x-axis corresponds to time, and the y-axis represents the spatial domain. High energy motion indicates more rapid or larger movements in the video, reflecting the speed and amplitude of the movements. For efficieny, we use [Pymoten's Pyramid Object](https://github.com/gallantlab/pymoten/blob/main/moten/pyramids.py) to compute the motion energy map.

In [ ]:
nimages = 150
stimulus_fps = 30
video_file = 'data/external/contemporary_001.mp4'
luminance_images = moten.io.video2luminance(video_file, nimages=nimages)
nimages, vdim, hdim = luminance_images.shape
pyramid = moten.pyramids.MotionEnergyPyramid(stimulus_vhsize=(vdim, hdim),
                                             stimulus_fps=stimulus_fps)
features = pyramid.project_stimulus(luminance_images)
print(pyramid)

In [ ]:
fig, ax = plt.subplots(figsize=(24, 4))
ax.matshow(features, aspect='auto')
ax.set_xlim(0,len(kd)) #disregards excess features layers
ax.set_xticks(range(0,len(kd),1)) #every second
plt.show()

### Spatial Components of the Motion Energy Filters

In the code snippet below, we demonstrate how the spatial filters vary in term of spatial frequencies, locations, and directions. (Source: [Pymoten's Documentation](https://gallantlab.org/pymoten/auto_examples/introduction/demo_gabor_filters.html)). For more details about the plotting process, please refer to the code in [/src/data/collection.py](https://github.com/kayesokua/gestures/blob/main/src/data/lotting.py).

In [ ]:
from src.data.plotting import plot_spatial_gabors
seconds = [t for t in range(60, 1030, 30)]
plot_spatial_gabors(pyramid, [s for s in seconds[0:10]])
plot_spatial_gabors(pyramid, [s for s in seconds[10:20]])
plot_spatial_gabors(pyramid, [s for s in seconds[20:30]])

## Observations
* The motion energy map indicates high energy motion throughout the video, with a color scale representing the magnitude of motion energy at each location.
* The direction of the Gabor filters throughout the frames (representing the timing) tells us that the spatial pattern is energetic by the second, with directions found in horizontal, vertical and diagonal directions.
* The luminance of the Gabor filter throughout the frames may indicate the changing lighting of the video collected, which is not relevant information about the performer's dance and can be ignored.
* The motion energy map may be more useful for comparative analysis rather than absolute motion analysis. However, it does require higher computing time and does not take music into account.
* A different motion energy map package or algorithm may be considered.

# Joint Velocities

This analysis describes the speed of the dancer's movements and how they relate to the tempo of the music. Examining joint velocities allows us to evaluate the performance from the viewpoints of timing and dynamics, connecting the energy of the dance to its timing.

In [ ]:
kd_vlab = ['Shoulder to Elbow','Elbow to Wrist','Hip to Knee','Ankle to Foot Index']
kd_vcols_r = ['v_right_shoulder_right_elbow', 'v_right_elbow_right_wrist','v_right_hip_right_knee','v_right_ankle_right_foot_index']
fig = plt.subplots(figsize=(24, 6))
for i, item in enumerate(kd_vcols_r):
    plt.plot(kd_v.index,kd_v[item], label=kd_vlab[i])
plt.xlim([2, 31])
plt.xticks(np.arange(2, 31, step=1))
plt.grid()
plt.title('Joint Velocity on Right Side')
plt.legend()
plt.show()

kd_vcols_l = ['v_left_shoulder_left_elbow', 'v_left_elbow_left_wrist','v_left_hip_left_knee','v_left_ankle_left_foot_index']
fig = plt.subplots(figsize=(24, 6))
for i, item in enumerate(kd_vcols_l):
    plt.plot(kd_v.index,kd_v[item], label=kd_vlab[i])
plt.xlim([2, 31])
plt.xticks(np.arange(2, 31, step=1))
plt.grid()
plt.title('Joint Velocity on Left Side')
plt.legend()
plt.show()

In [ ]:

kd_d = kd.filter(regex='^d_')

kd_v = kd.filter(regex='^v_')
kd_v.head()

# Beat Alignment

Beat Alignment: This metric measures how well the dancer's movements are synchronized with the music, and whether their movements align with the beats. This helps us assess the dancer's sense of timing and musicality, further exploring the temporal aspects of the performance.

In [13]:
from src.features.audio import extract_tempo_and_beats, extract_rms_energy, extract_zero_crossing_rate
tempo, beat_times = extract_tempo_and_beats(video_file_path)

rms = extract_rms_energy(video_file_path)
zcr = extract_zero_crossing_rate(video_file_path)

print(rms)
print(zcr)

if np.any(rms != 0):
    print("Array contains non-zero values")
else:
    print("Array does not contain non-zero values")
    
if np.any(zcr != 0):
    print("Array contains non-zero values")
else:
    print("Array does not contain non-zero values")


[[0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]]
Array contains non-zero values
Array contains non-zero values


# Principal Component Analysis

Principal Component Analysis (PCA) Visualization: PCA can be used to reduce the dimensionality of the kinematic data and project it onto a lower-dimensional space that can be easily visualized. This can help to identify the most important features that distinguish different dance styles or gestures.

Scatter plot trends that leans towards (positive,positive) indicates strong correlation. If a joint within the same group moves, most like the other joint is moving.  This ggives up insight on how to structure analysis of gestures.
 
PCA can provide insights into the structure and relationships among joint movements in a dance performance. It can help identify patterns of correlation and similarity among different joints, but it does not necessarily provide causal explanations for these patterns.

In [ ]:
beat_times = beat_times[2:32]

fig, axs = plt.subplots(1, 3, figsize=(24, 6))
axs[0].plot(kd.index, kd['beat_time'])
axs[1].plot(kd.index, rms)
axs[0].plot(kd.index, zcr)
plt.show()


In [ ]:
joints_upper = ['nose', 'forehead', 'mouth_right', 'mouth_left']
joints_mid = ['right_shoulder', 'left_shoulder', 'right_hip', 'left_hip']
joints_fingers = ['right_thumb', 'right_index', 'right_pinky', 'left_thumb', 'right_index', 'right_pinky']
joint_arms = ['right_shoulder', 'right_elbow', 'right_wrist', 'left_shoulder', 'left_elbow', 'left_wrist']
joint_lower = ['right_knee', 'right_ankle', 'right_foot_index', 'left_knee', 'left_ankle', 'left_foot_index']
all_joints = joints_upper + joints_mid + joints_fingers + joint_arms + joint_lower

fig, axs = plt.subplots(2, 3, figsize=(24, 12))
axs = axs.flatten()
joints_groups = [joints_upper, joints_mid, joints_fingers, joint_arms, joint_lower, all_joints]
joints_labels = ["Upper Body", "Mid Body", "Fingers", "Arms", "Legwork", "All Joints"]

for i, joint_group in enumerate(joints_groups):
    joint_data = []
    for joint in joint_group:
        joint_x, joint_y, joint_z = joint + "_x", joint + "_y", joint + "_z"
        np_joint = np.stack([kd[joint_x], kd[joint_y], kd[joint_z]], axis=1)
        joint_data.append(np_joint)

    np_joints = np.concatenate(joint_data, axis=1)
    scaler = StandardScaler()
    joints_std = scaler.fit_transform(np_joints)
    pca = PCA(n_components=2)
    principal_components = pca.fit_transform(joints_std)

    ax = axs[i]
    ax.scatter(principal_components[:, 0], principal_components[:, 1])
    ax.set_xlabel('Principal Component 1')
    ax.set_ylabel('Principal Component 2')
    ax.xaxis.set_major_locator(MultipleLocator(2.5))
    ax.yaxis.set_major_locator(MultipleLocator(2.5))
    ax.set_xlim(-12.5, 12.5)
    ax.set_ylim(-12.5, 12.5)
    ax.set_title(f"{joints_labels[i]}")
    ax.grid(True)

plt.tight_layout()
plt.show()

## Sources

1. https://link.springer.com/article/10.3758/s13428-021-01612-7
2. https://gallantlab.org/pymoten

In [ ]:
4. **Joint Velocities**: 
5. **Beat Alignment**: This metric measures how well the dancer's movements are synchronized with the music, and whether their movements align with the beats. This helps us assess the dancer's sense of timing and musicality, further exploring the temporal aspects of the performance.
6. **Principal Component Analysis**: This statistical technique allows us to describe the relationship between groups of joints in the dance. By analyzing the principal components, we can better understand the shape, space, and gesture aspects of the performance, synthesizing the insights gained from previous metrics.
